In [1]:
%matplotlib inline


词汇嵌入:编码词汇语义
===========================================

单词嵌入是真实数字的密集向量,在你的词汇表中每一个单词都是. 在NLP中, 通常情况下,
您的特性就是单词!但是你应该怎么在你的电脑中表示一个单词?可以存储它的ascii字符表示,
但那仅仅告诉你单词 *是* 什么,没有说太多它 *意味* 着什么 (你也许可以从它的词缀中派生出它的词性,
或者从它的大小写中得到它的属性,但并不多.). 更重要的是, 在什么意义上你能把这些表象结合起来? 
我们经常需要神经网络的密集输出, 输入为 $|V|$ 维, 其中 $V$ 是我们的词汇表, 
但经常输出是更小维度的 (如果我们只预测少量的标签的话).
我们如何从一个大的维度空间得到一个更小的维度空间?

如果我们不用 ascii 字符表示, 而使用one-hot encoding呢?
那就是, 我们用如下所示表示 $w$ 字符

\begin{align}\overbrace{\left[ 0, 0, \dots, 1, \dots, 0, 0 \right]}^\text{|V| elements}\end{align}

其中 1 是 $w$ 的特征位置.其他的单词也是在其他位置有一个1, 在另外的位置都是0. 

除了巨大的占用空间外,这种表述有一个巨大的缺陷. 它仅仅简单的把所有的单词都看作独立实体认
为它们彼此之间毫无关联.我们真正想要的是单词之间有一些 *相似* .为什么? 让我们来看一下例子.

假设我们正在搭建一个语言模型. 假设我们在训练集中看到了如下语句.

* The mathematician ran to the store.
* The physicist ran to the store.
* The mathematician solved the open problem.

现在假设我们得到了一个在训练集从未看到过的新句子:

* The physicist solved the open problem.

我们的语言模型对此句可能运行的不错, 但如果我们能使用以下两个事实,情况会好得多吗:

* 我们看到数学家和物理学家在句子中有着相同的作用.它们之间有一个语义关系.
* 我们已经看到数学家在这个新的没看过的的句子中扮演着同样的角色, 就像我们现在看到的物理学家一样.

然后我们就推断物理学家在这个句子里是很合适的?这就是我们指的相似的意思:我们指的是
*语义相似度*, 不仅仅是拼字一样的表示.
它是一种通过连接我们所看到的和我们没有看到的东西之间的点来对抗语言数据稀疏性的技术.
这个例子当然要依赖于一个基本的语言假设:在相似的语境中出现的单词在语义上是相互关联的. 
这被叫做 `distributional
hypothesis <https://en.wikipedia.org/wiki/Distributional_semantics>`__.


Getting Dense Word Embeddings(密集字嵌入)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

我们如何来解决那个问题?我们怎么能在单词中编码语义相似呢? 也许我们会想出一些语义属性.
举个例子, 我们看到了, 数学家和物理学家都会跑, 也许我们可以把"能跑"这个语义属性给一个高分.
考虑一下其他的属性, 想象一下, 你可能会在这些属性上给一些普通的单词得分.

如果每一个属性都是一维, 那我们可以给一个向量代表一个单词, 像这样:

\begin{align}q_\text{mathematician} = \left[ \overbrace{2.3}^\text{can run},
   \overbrace{9.4}^\text{likes coffee}, \overbrace{-5.5}^\text{majored in Physics}, \dots \right]\end{align}

\begin{align}q_\text{physicist} = \left[ \overbrace{2.5}^\text{can run},
   \overbrace{9.1}^\text{likes coffee}, \overbrace{6.4}^\text{majored in Physics}, \dots \right]\end{align}

这样我们就可以通过如下来得到这些单词之间的相似度:

\begin{align}\text{Similarity}(\text{physicist}, \text{mathematician}) = q_\text{physicist} \cdot q_\text{mathematician}\end{align}

尽管通常情况下需要归一化:

\begin{align}\text{Similarity}(\text{physicist}, \text{mathematician}) = \frac{q_\text{physicist} \cdot q_\text{mathematician}}
   {\| q_\text{\physicist} \| \| q_\text{mathematician} \|} = \cos (\phi)\end{align}

其中 $\phi$ 是两个向量的角度. 这就意味着,极端相似的单词(嵌入方向是同一个)
会得到相似度为1.反之为 -1.


你可以认为本章刚开始的稀疏one-hot 向量是我们刚定义向量的特殊形式,其中单词的相似度为 0, 
然后我们可以给每一个单词一些独特的语义属性.这些向量是 *密集的* , 也就是说他们是非零的.

但是这些新的向量是一种巨大的痛苦:你可以想到数千种不同的语义属性,它们可能与决定相似性有关,
而且究竟你怎样把它们设置成不同的属性? 深度学习的中心思想是比起需要程序员去自己设计特征,
神经网络学习特征的表示. 所以为什么不在我们的模型中让单词嵌入到系数中,然后让它们在训练中更新呢? 
这就是我们要做的. 我们会有一些 *潜在的语义属性* 网络可以, 严肃来讲, 学习. 注意, 嵌入词可能无法解释.
那就是尽管如上所示我们手工制作的矢量图,我们可以看到数学家和物理学家的相似之处是他们都喜欢咖啡.
如果我们允许神经网络学习嵌入, 并看到数学家和物理学家在第二个维度中有很大的价值,
但是它意味着什么很不清晰. 在潜在语义来讲它们是相似的, 但是对我们来说是无法解释的. 


总结一下, **单词嵌入是一个单词 *语义* 的表示,语义信息的有效编码可能与手头任务相关**. 
你也可以嵌入其他的东西: 部分的语音标签, 解析树, 其他任何东西! 特征嵌入是这个领域的核心思想.


Word Embeddings in Pytorch（Pytorch中的单词嵌入）
~~~~~~~~~~~~~~~~~~~~~~~~~~

在我们举例或练习之前, 关于如何在Pytorch中使用嵌入以及在一般的深度学习编程中,有一些快速
的说明.与制作one-hot向量时我们对每一个单词定义一个特别的索引相似,单词嵌入时同样需要对每
一个单词定义一个特别的索引. 这些将是查找表中的键. 意思是,嵌入被储存为一个 $|V| \times D$ 矩阵, 
其中 $D$ 是嵌入的维度, 这样的词被赋予了索引 $i$ 它的嵌入被储存在矩阵的
第 $i$ 行. 在所有的代码中, 从单词到索引的映射是一个命名的字典 word\_to\_ix.

允许你使用嵌入的模块式 torch.nn.Embedding,这需要两个参数:词汇量和嵌入的维度.

为了索引到这个表中,你需要使用 torch.LongTensor (索引为整数,不能为浮点数).




In [2]:
# 作者: Robert Guthrie

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [4]:
nn.Embedding?

In [13]:
word_to_ix = {"hello": 0, "world": 1,'me':2}
embeds = nn.Embedding(2, 5)  # 2 单词, 5 维嵌入
lookup_tensor = torch.LongTensor([word_to_ix["hello"]])
hello_embed = embeds(autograd.Variable(lookup_tensor))
print(hello_embed)

tensor([[-0.8330,  0.3568,  1.3209, -0.5273, -0.9138]])


例子: N-Gram 语言模型
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

回想一下 在n-gram语言模型中,给定一系列单词 $w$ , 我们需要计算

\begin{align}P(w_i | w_{i-1}, w_{i-2}, \dots, w_{i-n+1} )\end{align}

$w_i$ 是句子中第i个单词.

本例中, 我们将计算一些训练集的损失函数并且用反向传播更新系数.




In [15]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# 我们将使用 Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# 我们应该对输入进行标记,但是我们将忽略它
# 建造一系列元组.  每个元组 ([ word_i-2, word_i-1 ], 都是目标单词)
trigrams = [(test_sentence[i:i+CONTEXT_SIZE], test_sentence[i +CONTEXT_SIZE])
            for i in range(len(test_sentence) - CONTEXT_SIZE)]
# 输出前 3, 为了你看到他的各式
print(trigrams[:3])

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [18]:
vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


In [23]:
emb=nn.Embedding(2,10)

In [27]:
emb(autograd.Variable(torch.LongTensor([0,1])))

tensor([[-0.4629,  0.5493, -1.4471,  0.2785, -1.0307, -1.1284,  0.2359,
          0.1504, -0.9224,  0.5913],
        [ 0.0732, -1.4927,  1.1664,  0.2566,  0.7201, -0.4593,  0.2768,
          0.1331,  1.7475,  0.5126]])

        vocab_size=1200 单词本
        embedding_dim=10，嵌入维度
        context_size=2， 单词长度
* 使用EMbedding(单词本个数，嵌入维度)，单词长度*嵌入维度
* linear1，
* relu
* Linear2, 单词本，预测下一个对应的单词位置

In [33]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        #embeds 输出维度context_size * embedding_dim,每个单词对应一行。
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        #输入单词个数
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        #self.Liner 
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
#单词本个数，单词嵌入维度，2
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(1000):
    total_loss = torch.Tensor([0])
    for context, target in trigrams:

        # 步骤 1. 准备好进入模型的数据 (例如将单词转换成整数索引,并将其封装在变量中)
        context_idxs = [word_to_ix[w] for w in context]
        context_var = autograd.Variable(torch.LongTensor(context_idxs))

        # 步骤 2. 回调 *积累* 梯度. 在进入一个实例前,需要将之前的实力梯度置零
        model.zero_grad()

        # 步骤 3. 运行反向传播,得到单词的概率分布
        log_probs = model(context_var)

        # 步骤 4. 计算损失函数. (再次注意, Torch需要将目标单词封装在变量中)
        loss = loss_function(log_probs, autograd.Variable(
            torch.LongTensor([word_to_ix[target]])))

        # 步骤 5. 反向传播并更新梯度
        loss.backward()
        optimizer.step()

        total_loss += loss.data
    if epoch%10==0:
        print(epoch,'loss:',loss.data)
    losses.append(total_loss)
#print(losses)  # 在训练集中每次迭代损失都会减小!

0 loss: tensor(4.5298)
10 loss: tensor(4.2976)
20 loss: tensor(4.0705)
30 loss: tensor(3.8427)
40 loss: tensor(3.6113)
50 loss: tensor(3.3692)
60 loss: tensor(3.1166)
70 loss: tensor(2.8520)
80 loss: tensor(2.5769)
90 loss: tensor(2.2951)
100 loss: tensor(2.0130)
110 loss: tensor(1.7378)
120 loss: tensor(1.4801)
130 loss: tensor(1.2483)
140 loss: tensor(1.0506)
150 loss: tensor(0.8859)
160 loss: tensor(0.7513)
170 loss: tensor(0.6427)
180 loss: tensor(0.5540)
190 loss: tensor(0.4818)
200 loss: tensor(0.4231)
210 loss: tensor(0.3747)
220 loss: tensor(0.3343)
230 loss: tensor(0.3005)
240 loss: tensor(0.2718)
250 loss: tensor(0.2471)
260 loss: tensor(0.2256)
270 loss: tensor(0.2068)
280 loss: tensor(0.1903)
290 loss: tensor(0.1757)
300 loss: tensor(0.1627)
310 loss: tensor(0.1512)
320 loss: tensor(0.1409)
330 loss: tensor(0.1318)
340 loss: tensor(0.1235)
350 loss: tensor(0.1161)
360 loss: tensor(0.1095)
370 loss: tensor(0.1034)
380 loss: tensor(1.00000e-02 *
       9.7952)
390 loss: tenso

Exercise: Computing Word Embeddings: Continuous Bag-of-Words(练习: 计算单词嵌入: 连续单词包)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

连续单词包模型 (CBOW) 在NLP深度学习中使用的很频繁. 这个模型尝试去预测文中目标单词的
前后一些单词. 它有别于语言建模, 因为CBOW不是顺序的, 也不需要是概率性的.CBOW被用来快
速训练单词嵌入,而这些嵌入被用来初始化一些复杂模型的嵌入.通常情况下, 这被称为 *预训练嵌入* .
它几乎总是能帮助提升百分之几的性能.

CBOW模型如下所示.给定一个目标单词 $w_i$ 和 $N$ 
代表单词每一遍的滑窗距, $w_{i-1}, \dots, w_{i-N}$
和 $w_{i+1}, \dots, w_{i+N}$ , 将所有上下文词统称为 $C$ ,CBOW试图去最小化如下

\begin{align}-\log p(w_i | C) = -\log \text{Softmax}(A(\sum_{w \in C} q_w) + b)\end{align}


其中 $q_w$ 是单词 $w$ 的嵌入.

在Pytorch中通过填充下面的类来实现这个模型. 一些建议:

* 想好你需要定义的系数.
* 确保你知道每一步操作后的构造. 如果想要重构请使用 .view().




In [81]:
CONTEXT_SIZE = 2  # 左右各2个单词
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# 通过从 `raw_text` 得到一组单词, 进行去重操作
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))

In [90]:
class CBOW(nn.Module):

    def __init__(self,vocab_size,context_size,embedding_dim):
        super(CBOW,self).__init__()
        self.embeding=nn.Embedding(vocab_size,embedding_dim)
        self.Linear1=nn.Linear(context_size*embedding_dim,128)
        self.Linear2=nn.Linear(128,vocab_size)

    def forward(self, inputs):
        x=self.embeding(inputs).view((1,-1))
        x=self.Linear1(x)
        x=F.relu(x)
        x=self.Linear2(x)
        x=F.log_softmax(x,dim=1)
        return x

In [95]:
autograd.Variable(torch.LongTensor([y]))

tensor([ 16])

In [ ]:
# 创建模型并且训练. 这里有一些函数可以在使用模型之前帮助你准备数据
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)

model=CBOW(vocab_size,4,embedding_dim)
loss_f=nn.NLLLoss()
optimizer=optim.SGD(model.parameters(),lr=1e-4)
CONTEXT_SIZE=4

for epoch in range(10000):
    loss=[]
    for words,target in data:        
        x=make_context_vector(words,word_to_ix)
        y=word_to_ix[target]
        model.zero_grad()
        x_=model(x)
        loss=loss_f(x_,autograd.Variable(torch.LongTensor([y])))
        loss.backward()
        optimizer.step()
    if epoch%100==0:
        print(epoch,'loss:',loss.data)

0 loss: tensor(3.6178)
100 loss: tensor(3.4572)
200 loss: tensor(3.2895)
300 loss: tensor(3.1178)
400 loss: tensor(2.9454)
500 loss: tensor(2.7732)
600 loss: tensor(2.5967)
700 loss: tensor(2.4123)
800 loss: tensor(2.2151)
900 loss: tensor(2.0030)
1000 loss: tensor(1.7807)
1100 loss: tensor(1.5551)
1200 loss: tensor(1.3372)
1300 loss: tensor(1.1379)
1400 loss: tensor(0.9608)
1500 loss: tensor(0.8096)
1600 loss: tensor(0.6849)
1700 loss: tensor(0.5848)
1800 loss: tensor(0.5036)
1900 loss: tensor(0.4377)
2000 loss: tensor(0.3838)
2100 loss: tensor(0.3397)
2200 loss: tensor(0.3030)
2300 loss: tensor(0.2723)
2400 loss: tensor(0.2465)
2500 loss: tensor(0.2245)
2600 loss: tensor(0.2056)
2700 loss: tensor(0.1893)
2800 loss: tensor(0.1751)
2900 loss: tensor(0.1627)
3000 loss: tensor(0.1519)
3100 loss: tensor(0.1422)
3200 loss: tensor(0.1336)
3300 loss: tensor(0.1259)
3400 loss: tensor(0.1189)
3500 loss: tensor(0.1125)
3600 loss: tensor(0.1068)
3700 loss: tensor(0.1015)
3800 loss: tensor(1.0000

In [74]:
CONTEXT_SIZE = 2  # 左右各2个单词
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# 通过从 `raw_text` 得到一组单词, 进行去重操作
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(data[:5])

class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        #embeds 输出维度context_size * embedding_dim,每个单词对应一行。
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        #输入单词个数
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        #self.Liner 
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs
CONTEXT_SIZE=4

losses = []
loss_function = nn.NLLLoss()
#单词本个数，单词嵌入维度，2
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(1000):
    total_loss = torch.Tensor([0])
    for context, target in data:

        # 步骤 1. 准备好进入模型的数据 (例如将单词转换成整数索引,并将其封装在变量中)
        context_idxs = [word_to_ix[w] for w in context]
        context_var = autograd.Variable(torch.LongTensor(context_idxs))

        # 步骤 2. 回调 *积累* 梯度. 在进入一个实例前,需要将之前的实力梯度置零
        model.zero_grad()

        # 步骤 3. 运行反向传播,得到单词的概率分布
        log_probs = model(context_var)

        # 步骤 4. 计算损失函数. (再次注意, Torch需要将目标单词封装在变量中)
        loss = loss_function(log_probs, autograd.Variable(
            torch.LongTensor([word_to_ix[target]])))

        # 步骤 5. 反向传播并更新梯度
        loss.backward()
        optimizer.step()

        total_loss += loss.data
    if epoch%100==0:
        print(epoch,'loss:',loss.data)
    losses.append(total_loss)
#print(losses)  # 在训练集中每次迭代损失都会减小!

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]
0 loss: tensor(4.2250)
100 loss: tensor(2.1237)
200 loss: tensor(0.3551)
300 loss: tensor(0.1184)
400 loss: tensor(1.00000e-02 *
       6.4310)
500 loss: tensor(1.00000e-02 *
       4.2594)
600 loss: tensor(1.00000e-02 *
       3.1305)
700 loss: tensor(1.00000e-02 *
       2.4508)
800 loss: tensor(1.00000e-02 *
       2.0009)
900 loss: tensor(1.00000e-02 *
       1.6830)
